In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

In [ ]:
# Read the stock data from a CSV file
stocks = pd.read_csv('../sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

In [ ]:
data['year1ChangePercent']

In [ ]:
# Define a function to split a list into chunks of a given size
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
# Split the list of stock symbols into groups of 100 for batch API calls
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

# Define the columns for the final DataFrame
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [ ]:
# Create an empty DataFrame with the defined columns
final_dataframe = pd.DataFrame(columns = my_columns)

# Loop through each symbol group and make batch API calls to fetch stock data
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # Extract relevant data and append it to the final DataFrame
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
# Display the final DataFrame
final_dataframe

In [ ]:
# Sort the final DataFrame by 'One-Year Price Return' in descending order
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
# Select the top 50 stocks with the highest one-year price return
final_dataframe = final_dataframe[:50]
# Reset the index of the DataFrame
final_dataframe.reset_index(inplace = True)
# Display the final DataFrame
final_dataframe

In [ ]:
# Function to input the portfolio size
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")

    while True:
        try:
            val = float(portfolio_size)
            print(val)
            break
        except ValueError:
            print('Please enter a number.')
            portfolio_size = input('Enter the size of your portfolio: ')
            

In [ ]:
# Calculate the position size for each stock
position_size = float(portfolio_size) / len(final_dataframe.index)
# Calculate the number of shares to buy for each stock and store it in the DataFrame
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
# Display the final DataFrame with the 'Number of Shares to Buy' calculated
final_dataframe

In [ ]:
# Define columns for the High Quality Momentum DataFrame
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

# Create an empty DataFrame for High Quality Momentum (HQM) Strategy
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

# Loop through each symbol group and fetch stock data for the HQM Strategy
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        # Append the stock data to the HQM DataFrame
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )

hqm_dataframe

In [ ]:
# Define time periods for calculating momentum percentiles
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

# Loop through each row in the HQM DataFrame and each time period
for row in hqm_dataframe.index:
    for time_period in time_periods:
        # Define column names for change and percentile
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        # Check if the change value is None, and if so, set it to 0.0
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

# Loop through each row in the HQM DataFrame and each time period again
for row in hqm_dataframe.index:
    for time_period in time_periods:
        # Define column names for change and percentile
        change_col = f'{time_period} Price Return'
        # Calculate the percentile score for the change value using the score function
        percentile_col = f'{time_period} Return Percentile'
        # Calculate the percentile score for the change value using the score function
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
# Display the HQM DataFrame with updated percentile scores
hqm_dataframe

In [ ]:
# Calculate the HQM Score for each stock by taking the mean of momentum percentiles
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

In [ ]:
# Sort the DataFrame to select the top 50 stocks with the highest HQM Scores
# Reset the index for the selected stocks
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

In [ ]:
# Accept the portfolio size input using the portfolio_input function
portfolio_input()

In [ ]:
# Calculate the position size for each stock based on the portfolio size and the number of selected stocks
position_size = float(portfolio_size)/len(hqm_dataframe.index)
# Calculate the number of shares to buy for each stock and update the DataFrame
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
# Display the DataFrame with the calculated number of shares to buy
hqm_dataframe

In [ ]:
# Save to excel
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
writer.save()